In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense

# Define a custom layer for the Transformer encoder
class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        # Multi-head attention layer
        self.mha = MultiHeadAttention(num_heads, d_model)
        # Feed forward network, consisting of two dense layers
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),  # First dense layer with ReLU activation
            Dense(d_model)  # Second dense layer without activation
        ])

        # Layer normalization layers
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        # Dropout layers for regularization
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        # Apply multi-head attention on the input
        attn_output = self.mha(x, x, x)
        # Apply dropout to the attention output if in training mode
        attn_output = self.dropout1(attn_output, training=training)
        # Add the original input to the attention output and normalize
        out1 = self.layernorm1(x + attn_output)

        # Pass the normalized output through the feed forward network
        ffn_output = self.ffn(out1)
        # Apply dropout to the feed forward network output if in training mode
        ffn_output = self.dropout2(ffn_output, training=training)
        # Add the normalized output to the feed forward network output and normalize again
        out2 = self.layernorm2(out1 + ffn_output)

        # Return the final output of the encoder layer
        return out2


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense

In [ ]:
class TransformerEncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(TransformerEncoderLayer, self).__init__()
    self.mha = MultiHeadAttention(num_heads, d_model)
    self.ffn = tf.keras.Sequential([Dense(dff, activation='relu'), Dense(d_model)])
    self.layernorm1 = LayerNormalization(epsilon=1e-6)
    self.layernorm2 = LayerNormalization(epsilon=1e-6)
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    attn_output = self.mha(x, x, x)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)

In [ ]:
class TransformerDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.mha1 = MultiHeadAttention(num_heads, d_model)
        self.mha2 = MultiHeadAttention(num_heads, d_model)

        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.layernorm3 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

In [ ]:
def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
  attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
  attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                 target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                               input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                               target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask,
             look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

In [ ]:
for epoch in range(epochs):
    # Initialize the training step
    for (batch, (inp, tar)) in enumerate(dataset):
        # Training code here

In [ ]:
pip install transformers

In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
# Example sentences
sentences = ["I love this product!", "This is a bad product."]

# Tokenize sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="tf")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from transformers import TFBertModel, BertTokenizer

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
bert_model = TFBertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Example sentences for tokenization
sentences = ["This is a positive example.", "This is a negative example."]

# Tokenize the input sentences
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='tf')

# Define input layers
input_ids = Input(shape=(None,), dtype='int32', name="input_ids")
attention_mask = Input(shape=(None,), dtype='int32', name="attention_mask")

# Wrap the BERT model call in a Lambda layer to handle the assertion
bert_output = tf.keras.layers.Lambda(lambda x: bert_model(x[0], attention_mask=x[1]).last_hidden_state[:, 0, :])([input_ids, attention_mask])

# Add a classification layer on top
x = Dense(128, activation='relu')(bert_output)
output = Dense(1, activation='sigmoid')(x)

# Construct the final model
fine_tuned_model = Model(inputs=[input_ids, attention_mask], outputs=[output])

# Compile the model
fine_tuned_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Example labels for the sentences
labels = tf.constant([1, 0], dtype=tf.float32)  # 1 for positive, 0 for negative sentiment

# Train the model
fine_tuned_model.fit([inputs['input_ids'], inputs['attention_mask']], labels, epochs=3, batch_size=2)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
1/1 [==============================] - 23s 23s/step - loss: 0.7143 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 0s 114ms/step - loss: 0.7617 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 0s 120ms/step - loss: 0.6725 - accuracy: 0.5000


In [ ]:
import numpy as np

# Convert the BatchEncoding object to a dictionary of NumPy arrays
inputs_dict = {key: np.array(value) for key, value in inputs.items()}

# Convert labels to a NumPy array
labels = np.array(labels)

# Train the model
fine_tuned_model.fit(inputs_dict, labels, epochs=3, batch_size=32)

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


1/1 [==============================] - 33s 33s/step - loss: 0.7881 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 0s 142ms/step - loss: 0.5255 - accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 133ms/step - loss: 0.4427 - accuracy: 0.5000


In [ ]:
test_sentences = ["I am not sure about this product.", "Absolutely fantastic!"]
test_inputs = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="tf")

predictions = fine_tuned_model.predict({'input_ids': test_inputs['input_ids'], 'attention_mask': test_inputs['attention_mask']})

# Interpret the predictions
for sentence, prediction in zip(test_sentences, predictions):
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    print(f"Sentence: '{sentence}' - Sentiment: {sentiment}")

1/1 [==============================] - 10s 10s/step
Sentence: 'I am not sure about this product.' - Sentiment: Negative
Sentence: 'Absolutely fantastic!' - Sentiment: Negative
